## Describe your model -> fine-tuned LLaMA 2
By Matt Shumer (https://twitter.com/mattshumer_)

The goal of this notebook is to experiment with a new way to make it very easy to build a task-specific model for your use-case.

First, use the best GPU available (go to Runtime -> change runtime type)

To create your model, just go to the first code cell, and describe the model you want to build in the prompt. Be descriptive and clear.

Select a temperature (high=creative, low=precise), and the number of training examples to generate to train the model. From there, just run all the cells.

You can change the model you want to fine-tune by changing `model_name` in the `Define Hyperparameters` cell.

#Data generation step

Write your prompt here. Make it as descriptive as possible!

Then, choose the temperature (between 0 and 1) to use when generating data. Lower values are great for precise tasks, like writing code, whereas larger values are better for creative tasks, like writing stories.

Finally, choose how many examples you want to generate. The more you generate, a) the longer it takes and b) the more expensive data generation will be. But generally, more examples will lead to a higher-quality model. 100 is usually the minimum to start.

In [ ]:
prompt = "Um modelo que receba perguntas de usuários de um serviço de telecomunicações e responda com as soluções."
temperature = .4
number_of_examples = 15

Run this to generate the dataset.

In [ ]:
!pip install openai

In [ ]:
import os
import openai
import random
import time

openai.api_key = "YOUR KEY HERE"

def generate_example(prompt, prev_examples, temperature=.5):
    messages=[
        {
            "role": "system",
            "content": f"You are generating data which will be used to train a machine learning model.\n\nYou will be given a high-level description of the model we want to train, and from that, you will generate data samples, each with a prompt/response pair.\n\nYou will do so in this format:\n```\nprompt\n-----------\n$prompt_goes_here\n-----------\n\nresponse\n-----------\n$response_goes_here\n-----------\n```\n\nOnly one prompt/response pair should be generated per turn.\n\nFor each turn, make the example slightly more complex than the last, while ensuring diversity.\n\nMake sure your samples are unique and diverse, yet high-quality and complex enough to train a well-performing model.\n\nHere is the type of model we want to train:\n`{prompt}`"
        }
    ]

    if len(prev_examples) > 0:
        if len(prev_examples) > 10:
            prev_examples = random.sample(prev_examples, 10)
        for example in prev_examples:
            messages.append({
                "role": "assistant",
                "content": example
            })

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temperature,
        max_tokens=1354,
    )

    return response.choices[0].message['content']

# Generate examples
prev_examples = []
for i in range(number_of_examples):
    print(f'Generating example {i}')
    example = generate_example(prompt, prev_examples, temperature)
    prev_examples.append(example)
    time.sleep(20)

print(prev_examples)

We also need to generate a system message.

In [ ]:
prev_examples = ['prompt\n-----------\nUm modelo que receba perguntas de usuários de um serviço de telecomunicações e responda com as soluções.\n-----------\n\nresponse\n-----------\nComo posso ajudar você com o serviço de telecomunicações? Por favor, faça sua pergunta e eu ficarei feliz em fornecer uma solução.\n-----------',
 'prompt\n-----------\nUm modelo que receba perguntas de usuários de um serviço de telecomunicações e responda com as soluções.\n-----------\n\nresponse\n-----------\nOlá! Como posso ajudar você hoje com o seu serviço de telecomunicações? Estou aqui para fornecer as melhores soluções para suas dúvidas e problemas.\n-----------',
 'prompt\n-----------\nUm modelo que receba perguntas de usuários de um serviço de telecomunicações e responda com as soluções.\n-----------\n\nresponse\n-----------\nBem-vindo ao nosso serviço de atendimento ao cliente de telecomunicações! Estou aqui para responder a todas as suas perguntas e fornecer soluções eficazes para os seus problemas. Por favor, diga-me qual é a sua dúvida ou problema específico e eu farei o meu melhor para ajudar você.\n-----------',
 'prompt\n-----------\nUm modelo que receba perguntas de usuários de um serviço de telecomunicações e responda com as soluções.\n-----------\n\nresponse\n-----------\nOlá! Sou um assistente virtual especializado em telecomunicações. Estou aqui para ajudar você com qualquer pergunta ou problema relacionado ao seu serviço. Por favor, forneça-me mais detalhes sobre a sua situação para que eu possa oferecer a melhor solução possível.\n-----------',
 'prompt\n-----------\nUm modelo que receba perguntas de usuários de um serviço de telecomunicações e responda com as soluções.\n-----------\n\nresponse\n-----------\nBem-vindo ao nosso serviço de suporte de telecomunicações! Estou aqui para ajudar você a resolver qualquer problema ou dúvida que você possa ter. Por favor, descreva sua situação com o máximo de detalhes possível para que eu possa fornecer uma solução personalizada e eficaz. Estou ansioso para ajudar você!',
 'prompt\n-----------\nUm modelo que receba perguntas de usuários de um serviço de telecomunicações e responda com as soluções.\n-----------\n\nresponse\n-----------\nOlá! Sou um assistente virtual especializado em telecomunicações e estou aqui para ajudar você com qualquer dúvida ou problema que você possa ter. Por favor, descreva sua situação de forma clara e detalhada para que eu possa fornecer uma solução adequada. Estou à disposição para ajudar você a aproveitar ao máximo o seu serviço de telecomunicações.',
 'prompt\n-----------\nUm modelo que receba perguntas de usuários de um serviço de telecomunicações e responda com as soluções.\n-----------\n\nresponse\n-----------\nBem-vindo ao nosso serviço de atendimento ao cliente de telecomunicações! Estou aqui para responder a todas as suas perguntas e fornecer soluções eficazes para os seus problemas. Por favor, diga-me qual é a sua dúvida ou problema específico e eu farei o meu melhor para ajudar você. Seja sobre configurações, planos, cobranças ou qualquer outra coisa, estou aqui para ajudar você a obter a melhor experiência possível com o nosso serviço de telecomunicações.',
 'prompt\n-----------\nUm modelo que receba perguntas de usuários de um serviço de telecomunicações e responda com as soluções.\n-----------\n\nresponse\n-----------\nOlá! Como posso ajudar você hoje com o seu serviço de telecomunicações? Estou aqui para fornecer as melhores soluções para suas dúvidas e problemas. Seja sobre problemas de conexão, configurações do aparelho, cobranças ou qualquer outra coisa, estou aqui para ajudar você a resolver qualquer problema e garantir que você tenha uma experiência positiva com o nosso serviço. Por favor, me diga qual é a sua dúvida ou problema específico e eu farei o meu melhor para ajudar você.',
 'prompt\n-----------\nUm modelo que receba perguntas de usuários de um serviço de telecomunicações e responda com as soluções.\n-----------\n\nresponse\n-----------\nComo posso ajudar você com o serviço de telecomunicações? Por favor, faça sua pergunta e eu ficarei feliz em fornecer uma solução. Seja sobre problemas de sinal, configurações do aparelho, planos de serviço ou qualquer outra coisa, estou aqui para ajudar você a resolver qualquer problema e garantir que você tenha uma experiência positiva com o nosso serviço. Estou ansioso para ajudar você!',
 'prompt\n-----------\nUm modelo que receba perguntas de usuários de um serviço de telecomunicações e responda com as soluções.\n-----------\n\nresponse\n-----------\nOlá! Sou um assistente virtual especializado em telecomunicações. Estou aqui para ajudar você com qualquer pergunta ou problema relacionado ao seu serviço. Por favor, forneça-me mais detalhes sobre a sua situação para que eu possa oferecer a melhor solução possível. Seja sobre problemas de conexão, cobranças, configurações do aparelho ou qualquer outra coisa, estou aqui para ajudar você a aproveitar ao máximo o seu serviço de telecomunicações.',
 'prompt\n-----------\nUm modelo que receba perguntas de usuários de um serviço de telecomunicações e responda com as soluções.\n-----------\n\nresponse\n-----------\nBem-vindo ao nosso serviço de suporte de telecomunicações! Estou aqui para ajudar você a resolver qualquer problema ou dúvida que você possa ter. Por favor, descreva sua situação com o máximo de detalhes possível para que eu possa fornecer uma solução personalizada e eficaz. Estou ansioso para ajudar você a aproveitar ao máximo o seu serviço de telecomunicações. Seja sobre problemas de conexão, configurações do aparelho, cobranças ou qualquer outra coisa, estou aqui para ajudar você a resolver qualquer problema e garantir que você tenha uma experiência positiva com o nosso serviço.',
 'prompt\n-----------\nUm modelo que receba perguntas de usuários de um serviço de telecomunicações e responda com as soluções.\n-----------\n\nresponse\n-----------\nOlá! Sou um assistente virtual especializado em telecomunicações e estou aqui para ajudar você com qualquer dúvida ou problema que você possa ter. Por favor, descreva sua situação de forma clara e detalhada para que eu possa fornecer uma solução adequada. Estou à disposição para ajudar você a aproveitar ao máximo o seu serviço de telecomunicações. Seja sobre problemas de conexão, configurações do aparelho, cobranças ou qualquer outra coisa, estou aqui para ajudar você a resolver qualquer problema e garantir que você tenha uma experiência positiva com o nosso serviço.',
 'prompt\n-----------\nUm modelo que receba perguntas de usuários de um serviço de telecomunicações e responda com as soluções.\n-----------\n\nresponse\n-----------\nOlá! Sou um assistente virtual especializado em telecomunicações. Estou aqui para ajudar você com qualquer pergunta ou problema relacionado ao seu serviço. Por favor, forneça-me mais detalhes sobre a sua situação para que eu possa oferecer a melhor solução possível.\n-----------',
 'prompt\n-----------\nUm modelo que receba perguntas de usuários de um serviço de telecomunicações e responda com as soluções.\n-----------\n\nresponse\n-----------\nOlá! Sou um assistente virtual especializado em telecomunicações. Estou aqui para ajudar você com qualquer pergunta ou problema relacionado ao seu serviço. Por favor, forneça-me mais detalhes sobre a sua situação para que eu possa oferecer a melhor solução possível.\n-----------',
 'prompt\n-----------\nUm modelo que receba perguntas de usuários de um serviço de telecomunicações e responda com as soluções.\n-----------\n\nresponse\n-----------\nBem-vindo ao nosso serviço de atendimento ao cliente de telecomunicações! Estou aqui para responder a todas as suas perguntas e fornecer soluções eficazes para os seus problemas. Por favor, diga-me qual é a sua dúvida ou problema específico e eu farei o meu melhor para ajudar você.\n-----------']

In [ ]:
def generate_system_message(prompt):

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
          {
            "role": "system",
            "content": "You will be given a high-level description of the model we are training, and from that, you will generate a simple system prompt for that model to use. Remember, you are not generating the system message for data generation -- you are generating the system message to use for inference. A good format to follow is `Given $INPUT_DATA, you will $WHAT_THE_MODEL_SHOULD_DO.`.\n\nMake it as concise as possible. Include nothing but the system prompt in your response.\n\nFor example, never write: `\"$SYSTEM_PROMPT_HERE\"`.\n\nIt should be like: `$SYSTEM_PROMPT_HERE`."
          },
          {
              "role": "user",
              "content": prompt.strip(),
          }
        ],
        temperature=temperature,
        max_tokens=500,
    )

    return response.choices[0].message['content']

system_message = generate_system_message(prompt)

print(f'The system message is: `{system_message}`. Feel free to re-run this cell if you want a better result.')

The system message is: `Dadas as perguntas dos usuários sobre o serviço de telecomunicações, você fornecerá soluções e respostas.`. Feel free to re-run this cell if you want a better result.


Now let's put our examples into a dataframe and turn them into a final pair of datasets.

In [ ]:
import pandas as pd

# Initialize lists to store prompts and responses
prompts = []
responses = []

# Parse out prompts and responses from examples
for example in prev_examples:
  try:
    split_example = example.split('-----------')
    prompts.append(split_example[1].strip())
    responses.append(split_example[3].strip())
  except:
    pass

# Create a DataFrame
df = pd.DataFrame({
    'prompt': prompts,
    'response': responses
})

# Remove duplicates
df = df.drop_duplicates()

print('There are ' + str(len(df)) + ' successfully-generated examples. Here are the first few:')

df.head()

There are 12 successfully-generated examples. Here are the first few:


,prompt,response
0,Um modelo que receba perguntas de usuários de ...,Como posso ajudar você com o serviço de teleco...
1,Um modelo que receba perguntas de usuários de ...,Olá! Como posso ajudar você hoje com o seu ser...
2,Um modelo que receba perguntas de usuários de ...,Bem-vindo ao nosso serviço de atendimento ao c...
3,Um modelo que receba perguntas de usuários de ...,Olá! Sou um assistente virtual especializado e...
4,Um modelo que receba perguntas de usuários de ...,Bem-vindo ao nosso serviço de suporte de telec...


Split into train and test sets.

In [ ]:
# Split the data into train and test sets, with 90% in the train set
train_df = df.sample(frac=0.9, random_state=42)
test_df = df.drop(train_df.index)

# Save the dataframes to .jsonl files
train_df.to_json('train.jsonl', orient='records', lines=True)
test_df.to_json('test.jsonl', orient='records', lines=True)

# Install necessary libraries

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Define Hyperparameters

In [ ]:
model_name = "NousResearch/llama-2-7b-chat-hf" # use this if you have access to the official LLaMA 2 model "meta-llama/Llama-2-7b-chat-hf", though keep in mind you'll need to pass a Hugging Face key argument
dataset_name = "/content/train.jsonl"
new_model = "llama-2-7b-custom"
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./results"
num_train_epochs = 1
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 5
max_seq_length = None
packing = False
device_map = {"": 0}

#Load Datasets and Train

In [ ]:
# Load datasets
train_dataset = load_dataset('json', data_files='train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='test.jsonl', split="train")

# Preprocess datasets
train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=5  # Evaluate every 20 steps
)
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,  # Pass validation dataset here
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)
trainer.train()
trainer.model.save_pretrained(new_model)

# Cell 4: Test the model
logging.set_verbosity(logging.CRITICAL)
prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\nEstou com problemas com o meu telefone. [/INST]" # replace the command here with something relevant to your task
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(prompt)
print(result[0]['generated_text'])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


[INST] <<SYS>>
Dadas as perguntas dos usuários sobre o serviço de telecomunicações, você fornecerá soluções e respostas.
<</SYS>>

Estou com problemas com o meu telefone. [/INST]  Peço desculpas pelo problema com o seu telefone. Posso ajudá-lo? Qual é o problema específico que está enfrentando? Temperos de chamadas, mensagens de texto, internet ou algo else?


#Run Inference

In [ ]:
from transformers import pipeline

Question = 'A minha tv está com uma qualidade horrível'
prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n{Question} [/INST]" # replace the command here with something relevant to your task
num_new_tokens = 400  # change to the number of new tokens you want to generate

# Count the number of tokens in the prompt
num_prompt_tokens = len(tokenizer(prompt)['input_ids'])

# Calculate the maximum length for the generation
max_length = num_prompt_tokens + num_new_tokens

gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=max_length)
result = gen(prompt)
print(f"Question: {Question}")
print(f"Response:{result[0]['generated_text'].replace(prompt, '')}")

Question: A minha tv está com uma qualidade horrível
Response:  Peço desculpas pelo problema com a qualidade da sua TV. Há várias razões pelas quais a qualidade da sua TV pode estar diminuindo. Aqui estão algumas possíveis causas e soluções:

1. Malha de TV: Se a sua TV está com uma malha de TV velha ou de qualidade baixa, é possível que a qualidade da imagem esteja diminuindo. Considera substituir a malha de TV por uma nova para melhorar a qualidade da imagem.
2. Conectores de TV: Se os conectores de TV estão danificados ou não estão conectados corretamente, pode estar causando problemas na qualidade da imagem. Tente verificar os conectores e repará-los se necessário.
3. Cable de TV: Se o cable de TV está danificado ou não estão conectados corretamente, pode estar causando problemas na qualidade da imagem. Tente verificar o cable e repará-lo se necessário.
4. Equipamento de TV: Se o equipamento de TV está velho ou de qualidade baixa, pode estar causando problemas na qualidade da image